# CAPSTONE

To create a recommender model to assist the Sales & Marketing Department in identifying cross-selling opportunities to our customers through identifying customer segments by applying clustering techniques on customer sales data, including natural language processing of product descriptions to identify product categories.

Deliverables: to deliver a recommender model that improves by at least 10% accuracy from baseline model without feature engineering.

Customer Segementation by 
- RFM: (Recency, Frequency, Monetary) using K-means/DBScan/Hierarchical clustering
https://www.optimove.com/resources/learning-center/customer-segmentation-via-cluster-analysis#:~:text=In%20the%20context%20of%20customer,archetypes%E2%80%9D%20or%20%E2%80%9Cpersonas%E2%80%9D.

https://www.analyticsvidhya.com/blog/2020/10/quick-guide-to-evaluation-metrics-for-supervised-and-unsupervised-machine-learning/

https://www.shopify.com.sg/encyclopedia/customer-segmentation#:~:text=Customer%20segmentation%20is%20the%20process,Number%20of%20employees


https://analyticsindiamag.com/how-to-measure-the-success-of-a-recommendation-system/

https://www.kaggle.com/code/azizozmen/customer-segmentation-cohort-rfm-analysis-k-means

https://www.kaggle.com/code/ozlemilgun/customer-segmentation-using-rfm-analysis

https://www.kaggle.com/code/mathchi/business-problem-with-customer-segmentation

- Product segmentation (categorical) using K modes, Jacard Distance
Categorize products into main categories by performing NLP on product description (parts of speech tagging)

https://www.analyticsvidhya.com/blog/2021/06/kmodes-clustering-algorithm-for-categorical-data/#:~:text=KModes%20clustering%20is%20one%20of,similar%20our%20data%20points%20are.

- Clustering of combination of categorical and numerical features (Gower Distance, Dimensionality Reduction Techniques, K-prototypes)

Dataset only has 8 columns. Segmentation would require feature engineering to identify additional features for clustering. 

Unique product descriptions are about 4000+, which detail product type and variants like colour, design, and number of items it contains.

We would use RFM (Recency, Frequency, Monetary) Analysis to numerical sales data 

8 feature columns, 540K observations
'InvoiceNo' - 25900 unique
'stockcode_caps' - 4070 unique
'description_new' - 4223 unique, includes adjustment entries like stock returns, discounts 
'Quantity'
'InvoiceDate' - 1 year period
'UnitPrice'
'CustomerID' - 4372 unique
'Country' - 38 countries, 91% from one country

Assume average gross margin is 42%, net margin is 7% - online retail
https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/margin.html

https://en.wikipedia.org/wiki/List_of_countries_by_United_Nations_geoscheme

1. What is your problem statement?  What will you actually be doing?

2. Who is your audience?  Why will they care?
 
3. What is your success metric?  How will you know if you are actually solving the problem in a useful way?

4. What is your data source?  What format is your data in?  How much cleaning and munging will be required?

5. What are potential challenges or obstacles and how will you mitigate them?

6. Is this a reasonable project given the time constraints that you have?



Week starting 25 Apr
First half
- Import and clean data
- NLP of product categories
- EDA on product categories
- Product category clustering - K modes
- Create basic recommender model, error analysis, tuning

Second half
- Feature engineering for sales data - RFM analysis
- RFM clustering (K-means)
- Add to recommender model, error analysis, tuning

Week starting 2 May
First half
- Clustering with combined categorical variables and numerical variables
- Add other clustering methods DBScan/ Hierarchical clustering/Gower Distance/ Dimensionality Reduction Techniques/ K-prototypes
- Error analysis, tuning

Second half
Prepare presentation slides
Stretch goal - Impute COGS based on industry assumptions to add dimension to the business proposal

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

import wordcloud
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet, fcluster
from scipy.spatial.distance import pdist
from sklearn.metrics.pairwise import cosine_similarity
from sklearn_extra.cluster import KMedoids
import funk_svd

In [2]:
# Set chart style
plt.style.use('Solarize_Light2')

In [3]:
# Set float display options
pd.set_option('display.float_format','{:,.2f}'.format)

In [4]:
# Import data
sales = pd.read_csv('../data/processed_sales.csv', 
                         dtype={'invoiceno': 'object', 
                                 'stockcode_caps': 'object', 
                                 'quantity': np.int32, 
                                 'unitprice': np.float32, 
                                 'customerid': 'object'
                                })

In [5]:
customer_location = pd.read_csv('../data/processed_customer.csv')
stock_descriptions= pd.read_csv('../data/cleaned_stock_descriptions.csv')
invoice = pd.read_csv('../data/cleaned_invoice.csv')

In [6]:
# Check column types and null values
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536493 entries, 0 to 536492
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   invoiceno       536493 non-null  object 
 1   stockcode_caps  536493 non-null  object 
 2   quantity        536493 non-null  int32  
 3   unitprice       536493 non-null  float32
 4   customerid      536493 non-null  object 
dtypes: float32(1), int32(1), object(3)
memory usage: 16.4+ MB


In [7]:
# Check column types and null values
customer_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5772 entries, 0 to 5771
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerid  5772 non-null   object
 1   domestic    5772 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 50.9+ KB


In [8]:
# Check column types and null values
stock_descriptions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3817 entries, 0 to 3816
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   stockcode_caps   3817 non-null   object
 1   description_new  3817 non-null   object
dtypes: object(2)
memory usage: 59.8+ KB


In [9]:
# Check column types and null values
invoice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23195 entries, 0 to 23194
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   invoiceno    23195 non-null  object
 1   invoicedate  23195 non-null  object
 2   test         23195 non-null  bool  
dtypes: bool(1), object(2)
memory usage: 385.2+ KB


In [10]:
# convert time to datetime
invoice['invoicedate'] = pd.to_datetime(invoice['invoicedate'])

In [11]:
# Join datasets together
sales_data = sales.merge(customer_location, on='customerid') \
            .merge(invoice, on='invoiceno')

In [12]:
# Calculate revenue from unit price and quantity
sales_data['revenue'] = sales_data['unitprice'] * sales_data['quantity'] 

In [13]:
# create date features
sales_data['yearmonth'] = sales_data['invoicedate'].dt.to_period('M')

In [14]:
# create date features
sales_data['quarter'] = sales_data['invoicedate'].dt.to_period('Q')

In [1]:
# create date features
sales_data['week'] = sales_data['invoicedate'].dt.to_period('W')

NameError: name 'sales_data' is not defined

### Feature engineering

Exponential decay function, assume effect of relevance decays uniformly over time. 

In [ ]:
sales_data['invoice_age'] = pd.to_timedelta(sales_data['invoicedate'].max() 
                                            - sales_data['invoicedate']) / np.timedelta64(1, "D")

In [ ]:
age_midpoint = sales_data['invoice_age'].max()/2

In [ ]:
sales_data['decay'] = np.exp(np.log(0.5) / age_midpoint * sales_data['invoice_age'])

In [ ]:
sales_data['decay_3M'] = np.exp(np.log(0.5) / 90 * sales_data['invoice_age'])

In [ ]:
sales_data['revenue_decay'] = sales_data['revenue'] * sales_data['decay']

In [ ]:
sales_data['revenue_decay_3M'] = sales_data['revenue'] * sales_data['decay_3M']

### Train Test split

In [ ]:
sales_data[sales_data['test']==True].groupby(['invoiceno', 'invoicedate'])['revenue'].sum()

In [ ]:
test_invoice_rev = sales_data[sales_data['test']==True].groupby(['invoicedate']).agg(
        total_revenue=('revenue',np.sum))

In [ ]:
test_invoice_rev['total_revenue'].sum()

In [ ]:
test_invoice_rev

In [ ]:
plt.plot(test_invoice_rev.index ,test_invoice_rev['total_revenue'] )

Customer grouping by products purchased

We have a sparse matrix with thousands of features, meaning we have encountered the curse of dimensionality. Without applying dimensionality reduction techniques, we would not be able to identify meaningful clusters in our data.

We would use Singular-Value Decomposition (SVD) because it works better for sparse data.

As the data would be normalized along the index axis first,  

We would do the customer product groupings for train invoices before Nov 2011, and only do matching for test invoices in Nov-Dec 2011, to prevent data leakage


In [ ]:
revenue_sum = sales_data[sales_data['invoicedate'].dt.strftime('%Y-%m-%d') < '2011-11-01'] \
        .groupby('customerid').agg(total_revenue=('revenue',np.sum)).reset_index()

In [ ]:
customer_positive = revenue_sum[revenue_sum['total_revenue']>0]

In [ ]:
customer_positive_data = sales_data[sales_data['customerid'].isin(customer_positive['customerid'])]

In [ ]:
X = pd.crosstab(index=customer_positive_data['customerid'],
            columns=customer_positive_data['stockcode_caps'],
            values=customer_positive_data['revenue'], 
            aggfunc=sum,
            normalize=True,
           ) #change normalization method to index???

In [ ]:
X = X.astype('float32')

In [ ]:
X.info()

In [ ]:
sc_svd = StandardScaler()
X_sc = sc_svd.fit_transform(X)

In [ ]:
truncatedSVD = TruncatedSVD(
    n_components=100, 
    algorithm='randomized', 
    n_iter=20, 
    random_state=42)

In [ ]:
X_truncated = truncatedSVD.fit_transform(X)

In [ ]:
km = KMeans(n_clusters=10, random_state=42)
km.fit(X_truncated)

In [ ]:
X['cluster_svd_km'] = km.labels_

In [ ]:
X['cluster_svd_km'].value_counts()

In [ ]:
dbscan = DBSCAN(eps=0.1)
dbscan.fit(X_truncated);

In [ ]:
X['cluster_svd_dbscan'] = dbscan.labels_

In [ ]:
X['cluster_svd_dbscan'].value_counts()

In [ ]:
z = linkage(X_truncated, 'single')

In [ ]:
X['cluster_svd_linkage'] = fcluster(z,0.01,criterion='distance')

In [ ]:
X['cluster_svd_linkage'].value_counts()

In [ ]:
sim_matrix = cosine_similarity(X)

In [ ]:
customer_sim = pd.DataFrame(sim_matrix, columns=X.index, index=X.index)
customer_sim

In [ ]:
kmedoids = KMedoids(n_clusters=5, random_state=42).fit(X)

In [ ]:
kmedoids.labels_.shape

In [ ]:
svc_kmedoids_cluster = pd.DataFrame(kmedoids.labels_, index=X.index, columns=['svc_kmedoids_cluster'])

In [ ]:
svc_kmedoids_cluster.value_counts()

In [ ]:
sales_data = sales_data.merge(svc_kmedoids_cluster, on='customerid', how='left')

In [ ]:
sales_data

In [ ]:
sales_data['svc_kmedoids_cluster'].fillna(value=-1,inplace=True)

In [ ]:
# set NaN as -1
sales_data[sales_data['svc_kmedoids_cluster']==-1]['revenue'].sum()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

sales_data.groupby('svc_kmedoids_cluster').agg(total_revenue=('revenue',np.sum),
                                                total_qty=('quantity',np.sum),
                                                num_invoices=('invoiceno',lambda x: x.nunique()),
                                                 unique_products=('stockcode_caps',lambda x: x.nunique()),
                                                cust_count=('customerid',lambda x: x.nunique())
                                               )

In [ ]:
sales_data['revenue'].sum()

In [ ]:
# Top 5 products in each group
for i in [-1,0,1,2,3,4,5]:
    print(sales_data[sales_data['svc_kmedoids_cluster']==i] \
            .groupby('stockcode_caps')['revenue'].sum() \
            .sort_values(ascending=False).head(5))

Basic Model

In [ ]:
# split train and test data
train_data = sales_data[sales_data['test'] == False].copy()

In [ ]:
test_data = sales_data[sales_data['test'] == True].copy()

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
test_data[test_data['customerid'].apply(len)==5]['invoicedate'].describe()

In [ ]:
#test_data[test_data['customerid'].apply(month)]
#pd.DatetimeIndex(test_data['customerid']).month

In [ ]:
test_data[test_data['customerid'].apply(len)==5]['invoicedate'].describe()

In [ ]:
test_invoices = sales_data[sales_data['test'] == True].groupby(['invoiceno',
                                                                'customerid',
                                                               'invoicedate',
                                                               'svc_kmedoids_cluster']) \
                                                    .agg(num_products=('stockcode_caps',np.size),                                      
                                                   ).reset_index()

In [ ]:
test_invoices

In [ ]:
test_data['cust_stock'] = test_data['customerid'] + '/' + test_data['stockcode_caps']

Recommender Models

In [ ]:
test_invoices.shape[0]

In [ ]:
# for each invoice: 
predict_data = []
for i in test_invoices['invoiceno'].values:
    predict_invoice = []
    past_products = []
    test_customerid = test_invoices.loc[test_invoices['invoiceno'] == i,'customerid'].item()
    test_invoicedate = test_invoices.loc[test_invoices['invoiceno'] == i,'invoicedate'].item()
    test_numproducts = test_invoices.loc[test_invoices['invoiceno'] == i,'num_products'].item()
    
    # sums all previous invoices of each customerid
    past_cust_invoices = train_data.loc[(train_data['customerid']==test_customerid),
                       ['customerid','stockcode_caps','revenue']] \
                        .groupby(['customerid','stockcode_caps']) \
                        .sum().reset_index()
                                         
    # compare number of products in test invoice 
    # vs unique products with positive sales in all past invoices
    past_cust_purchases = past_cust_invoices[past_cust_invoices['revenue']>0]
    past_products.extend(past_cust_purchases['stockcode_caps'])
    
    if past_cust_purchases.shape[0] > 0:
    # if there were past purchases for this customer         
        invoice_count_difference = past_cust_purchases.shape[0] - test_numproducts       
        if invoice_count_difference >= 0:                                                 
        # and if the past purchases had more products than the test invoice
        # select the top n products where n = product length of test invoice
            predict_data.extend(past_cust_purchases.head(test_numproducts).values.tolist())
        
        else:
        # else, being that the past puchases were fewer than test invoice,
        # add the products from past purchases, 
            predict_data.extend(past_cust_purchases.values.tolist())
        # and add additional products from all past invoices
        # to make up for the shortfall in products
            past_all_invoices = train_data.loc[(train_data['invoicedate']<test_invoicedate),
                               ['stockcode_caps','revenue']] \
                                .groupby(['stockcode_caps']) \
                                .sum().reset_index()
            past_all_invoices['customerid'] = test_customerid
            past_all_purchases = past_all_invoices[past_all_invoices['revenue']>0]      
            past_all_purchases_select = past_all_purchases[~past_all_purchases['stockcode_caps'].isin(
                past_products)]
            predict_invoice.extend(past_all_purchases_select[['customerid',
                                                              'stockcode_caps',
                                                              'revenue']] \
                                .sort_values(by='revenue',ascending=False) \
                                .head(-invoice_count_difference).values.tolist())   
    else:
    # else, being that there were no past purchases 
    # select all sales before test invoice date
        past_all_invoices = train_data.loc[(train_data['invoicedate']<test_invoicedate),
                           ['stockcode_caps','revenue']] \
                            .groupby(['stockcode_caps']) \
                            .sum().reset_index()
        past_all_invoices['customerid'] = test_customerid
        past_all_purchases = past_all_invoices[past_all_invoices['revenue']>0]
        predict_invoice.extend(past_all_purchases[['customerid','stockcode_caps','revenue']] \
                            .sort_values(by='revenue',ascending=False) \
                            .head(test_numproducts).values.tolist())   
    predict_data.extend(predict_invoice)

In [ ]:
predict = pd.DataFrame(predict_data, columns=['customerid','stockcode_caps','revenue'])

In [ ]:
predict['cust_stock'] = predict['customerid'] + '/' + predict['stockcode_caps']

In [ ]:
test_data['prediction'] = test_data['cust_stock'].isin(predict['cust_stock'])

In [ ]:
test_data[test_data['prediction']==True]['revenue'].sum() / test_data['revenue'].sum()

Model with exponential decay

In [ ]:
# replace 'revenue' with 'revenue_decay' 
# for each invoice: 
predict_data = []
for i in test_invoices['invoiceno'].values:
    predict_invoice = []
    past_products = []
    test_customerid = test_invoices.loc[test_invoices['invoiceno'] == i,'customerid'].item()
    test_invoicedate = test_invoices.loc[test_invoices['invoiceno'] == i,'invoicedate'].item()
    test_numproducts = test_invoices.loc[test_invoices['invoiceno'] == i,'num_products'].item()
    
    # sums all previous invoices of each customerid
    past_cust_invoices = train_data.loc[(train_data['customerid']==test_customerid),
                       ['customerid','stockcode_caps','revenue_decay']] \
                        .groupby(['customerid','stockcode_caps']) \
                        .sum().reset_index()
                                         
    # compare number of products in test invoice 
    # vs unique products with positive sales in all past invoices
    past_cust_purchases = past_cust_invoices[past_cust_invoices['revenue_decay']>0]
    past_products.extend(past_cust_purchases['stockcode_caps'])
    
    if past_cust_purchases.shape[0] > 0:
    # if there were past purchases for this customer         
        invoice_count_difference = past_cust_purchases.shape[0] - test_numproducts       
        if invoice_count_difference >= 0:                                                 
        # and if the past purchases had more products than the test invoice
        # select the top n products where n = product length of test invoice
            predict_data.extend(past_cust_purchases.head(test_numproducts).values.tolist())
        
        else:
        # else, being that the past puchases were fewer than test invoice,
        # add the products from past purchases, 
            predict_data.extend(past_cust_purchases.values.tolist())
        # and add additional products from all past invoices
        # to make up for the shortfall in products
            past_all_invoices = train_data.loc[(train_data['invoicedate']<test_invoicedate),
                               ['stockcode_caps','revenue_decay']] \
                                .groupby(['stockcode_caps']) \
                                .sum().reset_index()
            past_all_invoices['customerid'] = test_customerid
            past_all_purchases = past_all_invoices[past_all_invoices['revenue_decay']>0]      
            past_all_purchases_select = past_all_purchases[~past_all_purchases['stockcode_caps'].isin(
                past_products)]
            predict_invoice.extend(past_all_purchases_select[['customerid',
                                                              'stockcode_caps',
                                                              'revenue_decay']] \
                                .sort_values(by='revenue_decay',ascending=False) \
                                .head(-invoice_count_difference).values.tolist())   
    else:
    # else, being that there were no past purchases 
    # select all sales before test invoice date
        past_all_invoices = train_data.loc[(train_data['invoicedate']<test_invoicedate),
                           ['stockcode_caps','revenue_decay']] \
                            .groupby(['stockcode_caps']) \
                            .sum().reset_index()
        past_all_invoices['customerid'] = test_customerid
        past_all_purchases = past_all_invoices[past_all_invoices['revenue_decay']>0]
        predict_invoice.extend(past_all_purchases[['customerid','stockcode_caps','revenue_decay']] \
                            .sort_values(by='revenue_decay',ascending=False) \
                            .head(test_numproducts).values.tolist())   
    predict_data.extend(predict_invoice)

In [ ]:
predict_decay = pd.DataFrame(predict_data, columns=['customerid','stockcode_caps','revenue'])

In [ ]:
predict_decay['cust_stock'] = predict_decay['customerid'] + '/' + predict_decay['stockcode_caps']

In [ ]:
test_data['prediction_decay'] = test_data['cust_stock'].isin(predict_decay['cust_stock'])

In [ ]:
test_data[test_data['prediction_decay']==True]['revenue'].sum() / test_data['revenue'].sum()

### Recent 3 months

In [ ]:
# replace 'revenue' with 'revenue_decay_3M' 
# for each invoice: 
predict_data = []
for i in test_invoices['invoiceno'].values:
    predict_invoice = []
    past_products = []
    test_customerid = test_invoices.loc[test_invoices['invoiceno'] == i,'customerid'].item()
    test_invoicedate = test_invoices.loc[test_invoices['invoiceno'] == i,'invoicedate'].item()
    test_startdate = test_invoicedate- timedelta(days=90) 
    test_numproducts = test_invoices.loc[test_invoices['invoiceno'] == i,'num_products'].item()
    
    # sums all previous invoices of each customerid
    past_cust_invoices = train_data.loc[(train_data['customerid']==test_customerid) &
                                        (train_data['invoicedate'] >= test_startdate) & 
                                        (train_data['invoicedate'] < test_invoicedate),
                       ['customerid','stockcode_caps','revenue_decay_3M']] \
                        .groupby(['customerid','stockcode_caps']) \
                        .sum().reset_index()
                                         
    # compare number of products in test invoice 
    # vs unique products with positive sales in all past invoices
    past_cust_purchases = past_cust_invoices[past_cust_invoices['revenue_decay_3M']>0]
    past_products.extend(past_cust_purchases['stockcode_caps'])
    
    if past_cust_purchases.shape[0] > 0:
    # if there were past purchases for this customer         
        invoice_count_difference = past_cust_purchases.shape[0] - test_numproducts       
        if invoice_count_difference >= 0:                                                 
        # and if the past purchases had more products than the test invoice
        # select the top n products where n = product length of test invoice
            predict_data.extend(past_cust_purchases.head(test_numproducts).values.tolist())
        
        else:
        # else, being that the past puchases were fewer than test invoice,
        # add the products from past purchases, 
            predict_data.extend(past_cust_purchases.values.tolist())
        # and add additional products from all past invoices
        # to make up for the shortfall in products
            past_all_invoices = train_data.loc[(train_data['invoicedate'] >= test_startdate) & 
                                (train_data['invoicedate']<test_invoicedate),
                               ['stockcode_caps','revenue_decay_3M']] \
                                .groupby(['stockcode_caps']) \
                                .sum().reset_index()
            past_all_invoices['customerid'] = test_customerid
            past_all_purchases = past_all_invoices[past_all_invoices['revenue_decay_3M']>0]      
            past_all_purchases_select = past_all_purchases[~past_all_purchases['stockcode_caps'].isin(
                past_products)]
            predict_invoice.extend(past_all_purchases_select[['customerid',
                                                              'stockcode_caps',
                                                              'revenue_decay_3M']] \
                                .sort_values(by='revenue_decay_3M',ascending=False) \
                                .head(-invoice_count_difference).values.tolist())   
    else:
    # else, being that there were no past purchases 
    # select all sales before test invoice date
        past_all_invoices = train_data.loc[(train_data['invoicedate'] >= test_startdate) &
                                            (train_data['invoicedate'] < test_invoicedate),
                                           ['stockcode_caps','revenue_decay_3M']] \
                                            .groupby(['stockcode_caps']) \
                                            .sum().reset_index()
        past_all_invoices['customerid'] = test_customerid
        past_all_purchases = past_all_invoices[past_all_invoices['revenue_decay_3M']>0]
        predict_invoice.extend(past_all_purchases[['customerid','stockcode_caps','revenue_decay_3M']] \
                            .sort_values(by='revenue_decay_3M',ascending=False) \
                            .head(test_numproducts).values.tolist())   
    predict_data.extend(predict_invoice)

In [ ]:
predict_decay3m_3mwindow = pd.DataFrame(predict_data, columns=['customerid','stockcode_caps','revenue'])

In [ ]:
predict_decay3m_3mwindow['cust_stock'] = predict_decay3m_3mwindow['customerid'] \
                                    + '/' + predict_decay3m_3mwindow['stockcode_caps']
test_data['predict_decay3m_3mwindow'] = test_data['cust_stock'].isin(predict_decay3m_3mwindow['cust_stock'])

In [ ]:
test_data[test_data['predict_decay3m_3mwindow']==True]['revenue'].sum() / test_data['revenue'].sum()

We have slight improvement

With customer clustering

In [ ]:
test_invoices.info()

In [ ]:
# replace 'revenue' with 'revenue_decay' 
# for each invoice: 
predict_data = []
for i in test_invoices['invoiceno'].values:
    predict_invoice = []
    past_products = []
    test_customerid = test_invoices.loc[test_invoices['invoiceno'] == i,'customerid'].item()
    test_invoicedate = test_invoices.loc[test_invoices['invoiceno'] == i,'invoicedate'].item()
    test_numproducts = test_invoices.loc[test_invoices['invoiceno'] == i,'num_products'].item()
    test_cluster = test_invoices.loc[test_invoices['invoiceno'] == i,'svc_kmedoids_cluster'].item()
    
    if test_cluster >= 0:  
        past_all_invoices = train_data.loc[(train_data['invoicedate']<test_invoicedate) &
                                           (train_data['svc_kmedoids_cluster']==test_cluster),
                           ['stockcode_caps','revenue_decay']] \
                            .groupby(['stockcode_caps']) \
                            .sum().reset_index()
        past_all_invoices['customerid'] = test_customerid
        past_all_purchases = past_all_invoices[past_all_invoices['revenue_decay']>0]
        predict_invoice.extend(past_all_purchases[['customerid','stockcode_caps','revenue_decay']] \
                            .sort_values(by='revenue_decay',ascending=False) \
                            .head(test_numproducts).values.tolist())   
    
    else:
    # else, being that there were no past purchases 
    # select all sales before test invoice date
        past_all_invoices = train_data.loc[(train_data['invoicedate']<test_invoicedate),
                           ['stockcode_caps','revenue_decay']] \
                            .groupby(['stockcode_caps']) \
                            .sum().reset_index()
        past_all_invoices['customerid'] = test_customerid
        past_all_purchases = past_all_invoices[past_all_invoices['revenue_decay']>0]
        predict_invoice.extend(past_all_purchases[['customerid','stockcode_caps','revenue_decay']] \
                            .sort_values(by='revenue_decay',ascending=False) \
                            .head(test_numproducts).values.tolist())   
    predict_data.extend(predict_invoice)

In [ ]:
predict_cluster = pd.DataFrame(predict_data, columns=['customerid','stockcode_caps','revenue'])

In [ ]:
predict_cluster['cust_stock'] = predict_cluster['customerid'] + '/' + predict_cluster['stockcode_caps']

In [ ]:
test_data['prediction_cluster'] = test_data['cust_stock'].isin(predict_cluster['cust_stock'])

In [ ]:
test_data[test_data['prediction_cluster']==True]['revenue'].sum() / test_data['revenue'].sum()

In [ ]:
test_data.head()

In [ ]:
test_data.groupby('svc_kmedoids_cluster', dropna=False).sum()

In [ ]:
train_data.loc[(train_data['invoicedate']<test_invoicedate),
                           ['stockcode_caps','revenue_decay']] \
                            .groupby(['stockcode_caps']) \
                            .sum().reset_index()

In [ ]:
test_data[test_data['customerid'].isin(
         test_data[test_data['prediction_cluster'] == -1]['customerid'].value_counts().index)
         ].groupby('customerid').sum().sort_values('prediction_cluster', ascending=False).head(10)



In [ ]:
test_data[test_data['svc_kmedoids_cluster'].isna()]

In [ ]:
train_data.loc[(train_data['invoicedate'].dt.strftime('%Y-%m-%d') < '2011-04-20'),
                           ['stockcode_caps','revenue_decay']] \
                            .groupby(['stockcode_caps']) \
                            .sum().reset_index()

In [ ]:
test_data[test_data['svc_kmedoids_cluster'] == True]['invoicedate'].describe()

In [ ]:
past_cust_invoices = train_data.loc[(train_data['svc_kmedoids_cluster']==1),
                           ['customerid','stockcode_caps','revenue_decay']] \
                            .groupby(['customerid','stockcode_caps']) \
                            .sum().reset_index().sort_values('revenue_decay', ascending=False)

In [ ]:
test_data[test_data['svc_kmedoids_cluster']==-1].head(10)

In [ ]:
past_cust_invoices.head(10)

In [ ]:
sales_data[sales_data['invoicedate'].dt.strftime('%Y-%m-%d') < '2011-11-01']

In [ ]:
null_cluster_top = test_data[test_data['svc_kmedoids_cluster']==-1].groupby(['stockcode_caps']) \
            .sum().sort_values('revenue_decay', ascending=False).head(20).index

In [ ]:
null_cluster_top

In [ ]:
stock_descriptions[stock_descriptions['stockcode_caps'].isin(null_cluster_top)]

Product classification

Extract Colours
Extract Numbers


1. Generate NGrams
2. Vectorize

In [ ]:
stop

In [ ]:
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet, fcluster
from scipy.spatial.distance import pdist

In [ ]:
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import regex as re
import webcolors

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

from scipy.cluster.hierarchy import dendrogram, linkage, cophenet, fcluster
from scipy.spatial.distance import pdist

In [ ]:
descriptions = revenue_codes[['description_new']].copy()

In [ ]:
color_list = list(webcolors.CSS3_NAMES_TO_HEX.keys())

In [ ]:
# Convert descriptions to lowercase
descriptions['description_lower'] = descriptions['description_new'].str.lower()

In [ ]:
descriptions

In [ ]:
type(stopwords.words('english'))

In [ ]:
def extract_non_text(string):
    extract = "".join(re.findall(r'[^a-zA-Z\s]',string))
    string = "".join(re.findall(r'[a-zA-Z\s]',string))
    return string, extract

In [ ]:
def extract_color(string_list):
    colors = []
    for i in color_list:
        if i in string_list:
            string_list.remove(i)
            colors.append(i)
    if len(colors) == 0:
        colors = np.nan
    return string_list, colors

In [ ]:
def extract_design_size(string_list):
    design_extracted = []
    size_extracted = []
    
    for i in design_list:
        if i in string_list:
            string_list.remove(i)
            design_extracted.append(i)
            
    for i in size_list:
        if i in string_list:
            string_list.remove(i)
            size_extracted.append(i)
    
    if len(design_extracted) == 0:
        design_extracted = np.nan
    if len(size_extracted) == 0:
        size_extracted = np.nan
    
    return string_list, design_extracted, size_extracted

In [ ]:
# Extract numerical and non-text characters
descriptions['clean'], descriptions['num_special_char'] = zip(*descriptions['description_lower'].
                                                              apply(extract_non_text))

In [ ]:
# Remove excess space and convert strings to lists of words
descriptions['clean'] = descriptions['clean'] \
                                            .str.strip() \
                                            .apply(lambda x: list(x.split()))

In [ ]:
# Extract colors
descriptions['clean'], descriptions['color']= zip(*descriptions['clean'].apply(extract_color))

In [ ]:
#descriptions['clean'].apply(lambda x: " ".join(x))

In [ ]:
# Combine back list of strings to one string each
X = descriptions['clean'].copy()

In [ ]:
X

In [ ]:
# Initial vectorization
cvec = CountVectorizer(ngram_range=(1, 2), tokenizer=lambda x: x, preprocessor=lambda x: x
                      )
cvec.fit(X)
X_cvec = cvec.transform(X)

In [ ]:
X_cvec = pd.DataFrame(X_cvec.todense(),
             columns=cvec.get_feature_names_out()
            )

In [ ]:
X_cvec.sum().sort_values(ascending=False).head(50)

In [ ]:
design_list = ['heart', 'vintage','christmas', 'rose', 'polkadot', 'design', 'flower', 
               'metal','glass', 'retrospot', 'crystal', 'love', ]
size_list = ['small', 'large', 'mini', 'assorted', 'set', 'of',
             'and', 'with', 'in', 'pack', 'w', 's', 'pair', 'bag']

In [ ]:
# Extract design and size related words
descriptions['clean2'], descriptions['design'], descriptions['size'] \
    = zip(*descriptions['clean'].apply(extract_design_size))

In [ ]:
descriptions[descriptions['description_lower'].str.contains('egg')]

In [ ]:
X = descriptions['clean2'].copy()

In [ ]:
# vectorization
cvec = CountVectorizer(ngram_range=(1, 2), tokenizer=lambda x: x, preprocessor=lambda x: x
                      )
cvec.fit(X)
X_cvec = cvec.transform(X)

X_cvec = pd.DataFrame(X_cvec.todense(),
             columns=cvec.get_feature_names_out()
            )

In [ ]:
X_cvec.sum().sort_values(ascending=False).head(50)

In [ ]:
sc_cvec = StandardScaler()
X_cvec_sc = sc_cvec.fit_transform(X_cvec)

In [ ]:
km = KMeans(n_clusters=20, random_state=42)
km.fit(X_cvec_sc)

In [ ]:
km.labels_

In [ ]:
revenue_codes['cluster_cvec_km'] = km.labels_

In [ ]:
revenue_codes['cluster_cvec_km'].value_counts()

In [ ]:
revenue_codes[revenue_codes['cluster_cvec_km']==42].sort_values(by='cluster_cvec_km')

In [ ]:
silhouette_score(X_cvec_sc, km.labels_)

In [ ]:
dbscan = DBSCAN(eps=0.1)
dbscan.fit(X_cvec_sc);

In [ ]:
silhouette_score(X_cvec_sc, dbscan.labels_)

In [ ]:
revenue_codes['cluster_cvec_dbscan'] = dbscan.labels_

In [ ]:
revenue_codes['cluster_cvec_dbscan'].value_counts()

In [ ]:
tvec = TfidfVectorizer(ngram_range=(1, 2), tokenizer=lambda x: x, preprocessor=lambda x: x
                      )
tvec.fit(X)
X_tvec = tvec.transform(X)

In [ ]:
X_tvec = pd.DataFrame(X_tvec.todense(),
             columns=tvec.get_feature_names_out()
            )

In [ ]:
X_tvec.sum().sort_values()

In [ ]:
sc_tvec = StandardScaler()
X_tvec_sc = sc_tvec.fit_transform(X_tvec)

In [ ]:
km_tvec = KMeans(n_clusters=20, random_state=42)
km_tvec.fit(X_tvec_sc)

In [ ]:
silhouette_score(X_tvec_sc, km_tvec.labels_)

In [ ]:
revenue_codes['cluster_tvec_km'] = km_tvec.labels_

In [ ]:
revenue_codes['cluster_tvec_km'].value_counts()

In [ ]:
dbscan_tvec = DBSCAN(eps=0.9)
dbscan_tvec.fit(X_tvec_sc);

In [ ]:
silhouette_score(X_tvec_sc, dbscan_tvec.labels_)

In [ ]:
revenue_codes['cluster_tvec_dbscan'] = dbscan.labels_

In [ ]:
revenue_codes['cluster_tvec_dbscan'].value_counts()

In [ ]:
X_cvec_hierarchy = X_cvec.values
z = linkage(X_cvec_hierarchy , 'single')

In [ ]:
dendrogram(z);

In [ ]:
fcluster(z,2,criterion='distance')

In [ ]:
revenue_codes['cluster_linkage'] = fcluster(z,1.2,criterion='distance')

In [ ]:
revenue_codes['cluster_linkage'].value_counts().loc[lambda x: x>1]

In [ ]:
revenue_codes[revenue_codes['cluster_linkage']==1077]

In [ ]:
silhouette_score(X_cvec_hierarchy, fcluster(z,1.2,criterion='distance'))

In [ ]:
X_tvec_hierarchy = X_tvec.values
z = linkage(X_tvec_hierarchy, 'single')

In [ ]:
dendrogram(z);

In [ ]:
revenue_codes['cluster_tvec_linkage'] = fcluster(z,0.9,criterion='distance')

In [ ]:
revenue_codes['cluster_tvec_linkage'].value_counts().loc[lambda x: x>1]

In [ ]:
silhouette_score(X_tvec_hierarchy, fcluster(z,0.9,criterion='distance'))

In [ ]:
revenue_codes[['stockcode_caps','description_new']].value_counts()

In [ ]:
revenue_codes['stockcode_caps_description'] = revenue_codes['stockcode_caps'] + revenue_codes['description_new']

In [ ]:
sales_data['stockcode_caps_description'] = sales_data['stockcode_caps'] + sales_data['description_new']

In [ ]:
sales_data = sales_data.merge(revenue_codes[['stockcode_caps_description','cluster_tvec_linkage']], 
                              how='left', 
                              on='stockcode_caps_description')

In [ ]:
sales_data.head()

In [ ]:
sales_data.pivot_table(index='customerid',columns='cluster_tvec_linkage',values='revenue', aggfunc=sum)

In [ ]:
product_by_customers = pd.crosstab(index=sales_data['customerid'],
            columns=sales_data['cluster_tvec_linkage'],
            values=sales_data['revenue'], 
            aggfunc=sum,
            normalize='index'
           )

In [ ]:
product_by_customers

In [ ]:
sc_product_by_customers = StandardScaler()
X_product_by_customers = sc_product_by_customers.fit_transform(product_by_customers.values)

In [ ]:
km_product_by_customers = KMeans(n_clusters=10, random_state=42)
km_product_by_customers.fit(X_product_by_customers)

In [ ]:
product_by_customers['cluster_cvec_km'] = km_product_by_customers.labels_

In [ ]:
product_by_customers['cluster_cvec_km'].value_counts()

In [ ]:
dbscan_product_by_customers = DBSCAN(eps=0.01)
dbscan_product_by_customers.fit(X_product_by_customers);

In [ ]:
product_by_customers['cluster_dbscan'] = dbscan_product_by_customers.labels_
product_by_customers['cluster_dbscan'].value_counts()

In [ ]:
#z = linkage(X_product_by_customers , 'single')
#dendrogram(z);

In [ ]:
#product_by_customers['cluster_linkage'] = fcluster(z, 0.9, criterion = 'distance')